## Cockpit API

In [1]:
# Import the CockpitATS client and required modules
import sys
import os
from datetime import datetime

# Add the parent directory to Python path to import clients module
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from clients.cockpit import CockpitATSClient

print("✅ Successfully imported CockpitATSClient")
print(f"Current time: {datetime.now()}")

# Note: CockpitATSClient requires an identity_provider_token parameter
# Example initialization (commented out since we don't have a real token):
# cockpit_client = CockpitATSClient("your_identity_provider_token_here")

print("⚠️  To use CockpitATSClient, you need to provide an identity_provider_token:")
print("   cockpit_client = CockpitATSClient('your_identity_provider_token')")

✅ Successfully imported CockpitATSClient
Current time: 2025-11-04 15:25:22.525226
⚠️  To use CockpitATSClient, you need to provide an identity_provider_token:
   cockpit_client = CockpitATSClient('your_identity_provider_token')


In [9]:
# Test CockpitATSClient (requires identity_provider_token)
print("Testing CockpitATSClient...")

# Example of how to test with a mock token (for demonstration purposes)
try:
    # This will fail because we need environment variables and a real token
    test_token = os.getenv("COCKPIT_KEY")
    cockpit_client = CockpitATSClient(test_token)
    
    # Test get_branch_details method
    print("\n🔍 Testing get_branch_details method...")
    try:
        branch_details = cockpit_client.get_branch_details()
        print("✅ Successfully retrieved branch details:")
        print(f"   Response type: {type(branch_details)}")
        if isinstance(branch_details, dict):
            print(f"   Number of fields: {len(branch_details)}")
            print(f"   Fields: {list(branch_details.keys())}")
        else:
            print(f"   Response: {branch_details}")
    except Exception as branch_error:
        print(f"❌ get_branch_details failed: {branch_error}")
        
except Exception as e:
    print(f"CockpitATSClient test failed: {e}")
    print("Make sure you have:")
    print("1. COCKPIT_BASE_URL environment variable set")
    print("2. A valid identity_provider_token (COCKPIT_KEY environment variable)")

Testing CockpitATSClient...

🔍 Testing get_branch_details method...
Error during Cockpit token exchange: 401 Client Error: Unauthorized for url: https://timing-prelive.recruitnowcockpit.nl/candidateprofile/api/authentication/token
❌ get_branch_details failed: 401 Client Error: Unauthorized for url: https://timing-prelive.recruitnowcockpit.nl/candidateprofile/api/authentication/token
Error during Cockpit token exchange: 401 Client Error: Unauthorized for url: https://timing-prelive.recruitnowcockpit.nl/candidateprofile/api/authentication/token
❌ get_branch_details failed: 401 Client Error: Unauthorized for url: https://timing-prelive.recruitnowcockpit.nl/candidateprofile/api/authentication/token


### Testing get_branch_details Method

The `get_branch_details()` method retrieves branch/owner information for the candidate from the Cockpit API endpoint: `candidatesprofile/owner`

In [ ]:
# Comprehensive test for get_branch_details method
import requests

def test_get_branch_details():
    """Test the get_branch_details method with various scenarios"""
    
    print("🧪 Testing get_branch_details method scenarios:")
    print("=" * 50)
    
    # Check if we have a valid cockpit_client
    if 'cockpit_client' not in globals():
        print("❌ No cockpit_client available. Run the previous cell first.")
        return
    
    # Test 1: Check if client is properly initialized
    print("1️⃣ Testing client initialization...")
    try:
        print(f"   Client base URL: {cockpit_client.base_url}")
        print(f"   Token available: {'Yes' if cockpit_client._bearer_token else 'No'}")
        print("   ✅ Client appears to be initialized")
    except Exception as e:
        print(f"   ❌ Client initialization issue: {e}")
        return
    
    # Test 2: Test get_branch_details method
    print("\n2️⃣ Testing get_branch_details API call...")
    try:
        branch_details = cockpit_client.get_branch_details()
        
        print("   ✅ API call successful!")
        print(f"   Response type: {type(branch_details)}")
        
        if isinstance(branch_details, dict):
            print(f"   Number of fields in response: {len(branch_details)}")
            
            if branch_details:
                print("   📋 Response structure:")
                for key, value in branch_details.items():
                    value_preview = str(value)[:50] + "..." if len(str(value)) > 50 else str(value)
                    print(f"      • {key}: {value_preview}")
            else:
                print("   ℹ️  Empty response (204 No Content)")
        else:
            print(f"   Response content: {branch_details}")
            
    except requests.exceptions.HTTPError as http_err:
        status_code = http_err.response.status_code if http_err.response else "unknown"
        print(f"   ❌ HTTP Error {status_code}: {http_err}")
        
        if status_code == 401:
            print("   💡 This likely means the identity_provider_token is invalid or expired")
        elif status_code == 403:
            print("   💡 This likely means the token doesn't have permission for this endpoint")
        elif status_code == 404:
            print("   💡 The endpoint might not exist or the candidate profile wasn't found")
            
    except Exception as e:
        print(f"   ❌ Unexpected error: {e}")
        print(f"   Error type: {type(e).__name__}")

# Run the test
test_get_branch_details()

🧪 Testing get_branch_details method scenarios:
1️⃣ Testing client initialization...
   Client base URL: https://timing-prelive.recruitnowcockpit.nl
   Token available: No
   ✅ Client appears to be initialized

2️⃣ Testing get_branch_details API call...
Error during Cockpit token exchange: 401 Client Error: Unauthorized for url: https://timing-prelive.recruitnowcockpit.nl/candidateprofile/api/authentication/token


NameError: name 'requests' is not defined

## Planbition

In [3]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import sys
import os

# Add the parent directory to Python path to import clients module
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from clients.planbition import PlanbitionClient

# Initialize the client (this will fail if environment variables are not set)
try:
    client = PlanbitionClient()
    print("✅ Successfully initialized PlanbitionClient")
except ValueError as e:
    print(f"❌ Failed to initialize PlanbitionClient: {e}")
    print("Make sure the following environment variables are set:")
    print("- PLANBITION_BASE_URL")
    print("- PLANBITION_KEY") 
    print("- PLANBITION_USERNAME")
    print("- PLANBITION_PASSWORD")
    client = None

✅ Successfully initialized PlanbitionClient


In [4]:
# Test the PlanbitionClient and get_schedule tool
print("Testing PlanbitionClient...")

if client is None:
    print("❌ Cannot test PlanbitionClient - client not initialized")
    print("Please ensure environment variables are properly configured")
else:
    # Test direct client call
    try:
        # Replace with actual employee number and dates for testing
        test_employee = "12345"  # Replace with a real employee number
        test_start = "2024-01-01"
        test_end = "2024-01-07"
        
        print(f"Testing direct client call for employee {test_employee}")
        schedule_data = client.get_employee_schedule(test_employee, test_start, test_end)
        print(f"Direct client result: {len(schedule_data) if isinstance(schedule_data, list) else 'Error'} records")
        
    except Exception as e:
        print(f"Direct client test failed: {e}")

Testing PlanbitionClient...
Testing direct client call for employee 12345
Direct client result: Error records
Direct client result: Error records
